# Shopify Fall 2022 Data Science Intern Challenge Q1

## Import data & library 

In [1]:
import pandas as pd
import os
os.chdir("/Users/ERIC/2022")
print(os.getcwd())
print("Setup complete.")

C:\Users\ERIC\2022
Setup complete.


In [2]:
shopify_data = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv", index_col=0)
shopify_data

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
1,53,746,224,2,cash,2017-03-13 12:36:56
2,92,925,90,1,cash,2017-03-03 17:38:52
3,44,861,144,1,cash,2017-03-14 4:23:56
4,18,935,156,1,credit_card,2017-03-26 12:43:37
5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...
4996,73,993,330,2,debit,2017-03-30 13:47:17
4997,48,789,234,2,cash,2017-03-16 20:36:16
4998,56,867,351,3,cash,2017-03-19 5:42:42


### a) Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.

#### First, I want to filter the data set into variables/columns that I am interested in. From there, I perform data analysis by using the describe() function to see the statistics of variables "order_amount" and "total_items"

In [3]:
data = shopify_data.loc[:, ['shop_id', 'order_amount', 'total_items']]
data

,shop_id,order_amount,total_items
order_id,,,
1,53,224,2
2,92,90,1
3,44,144,1
4,18,156,1
5,18,156,1
...,...,...,...
4996,73,330,2
4997,48,234,2
4998,56,351,3


#### Check that there are no missing values in our data set

In [4]:
data.isna().sum()

shop_id         0
order_amount    0
total_items     0
dtype: int64

#### Statistical analysis on our dataset

In [5]:
data[['order_amount','total_items']].describe()

,order_amount,total_items
count,5000.000000,5000.00000
mean,3145.128000,8.78720
std,41282.539349,116.32032
min,90.000000,1.00000
25%,163.000000,1.00000
50%,284.000000,2.00000
75%,390.000000,3.00000
max,704000.000000,2000.00000


#### From the table, we gain some important knowledge about the dataset: 
#### 1) The average order_amount is 3145.128, which was mentioned in the question as the AOV.
#### 2) The average total_items is 8.7872.
#### 3) The minimum total_items is 1, and maximum total_items is 2,000.
#### 4) The minimum order_amount is 90, and maximum order_amount is 704,000.
* The range between these two value suggest that there may exist potential outliers in the dataset. 

#### There may exist potential outliers from the statistics above. Using the groupby() and sort functions to find the average order amount per store, along with the total items sold (Arranged in descending order).

In [6]:
data_shop = data.groupby(['shop_id'])[['order_amount', 'total_items']].mean()
data_shop_arranged = data_shop.sort_values(by='order_amount', ascending = False)
data_shop_arranged

,order_amount,total_items
shop_id,,
42,235101.490196,667.901961
78,49213.043478,1.913043
50,403.545455,2.090909
90,403.224490,2.265306
38,390.857143,2.057143
...,...,...
53,214.117647,1.911765
100,213.675000,1.925000
32,189.976190,1.880952


#### From the table above, we can see that most shops are normal except for shop 42 and shop 78. 

#### To check the average shoe price for shop 42 by taking its order amount and divide it by its total items amount

In [7]:
shop_42 = data_shop_arranged.iloc[0]
shop_42_average = shop_42.order_amount/shop_42.total_items
print("The average price per shoe for shop 42 is: $", shop_42_average)

The average price per shoe for shop 42 is: $ 352.0


#### The average price per shoe in shop 42 is $352, which is acceptable assuming the shoe is unique and rare.

#### To check the average shoe price for shop 78 by taking its order amount and divide it by its total items amount

In [8]:
shop_78 = data_shop_arranged.iloc[1]
shop_78_average = shop_78.order_amount/shop_78.total_items
print("The average price per shoe for shop 78 is: $", shop_78_average)

The average price per shoe for shop 78 is: $ 25725.0


#### The average price per shoe in shop 78 is $25725. This is a very unusual amount and therefore it is considered an outlier and should not be considered in the AOV calculation. 

### b) What metric would you report for this dataset?
### c) What is its value?

#### A better way to calculate this AOV is to take the total order amount and divided by the total items purchased (excluding all the values from shop 78). Start by filtering out all the data from shop 78.

In [9]:
data_without_78 = data.loc[(data.shop_id != 78)]
data_without_78

,shop_id,order_amount,total_items
order_id,,,
1,53,224,2
2,92,90,1
3,44,144,1
4,18,156,1
5,18,156,1
...,...,...,...
4996,73,330,2
4997,48,234,2
4998,56,351,3


#### Using the dataset without shop 78 and find the total order amount and the total amount of items purchased

In [10]:
total_order_amount = data_without_78.order_amount.sum()
total_order_amount
total_items_puchased =  data_without_78.total_items.sum()
total_items_puchased
print("The total order amount = $", total_order_amount)
print("The total items purchased = ", total_items_puchased)

The total order amount = $ 13461840
The total items purchased =  43848


#### Divide the total order amount by the total items purchased.

In [11]:
aov = total_order_amount/total_items_puchased
print("The average order value = $",aov)

The average order value = $ 307.01149425287355


#### A better value for AOV would be $307.00